In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=23

In [0]:
df = pd.read_csv('ner_short.csv', index_col=0)
df.head()

,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [0]:
# number of sentences
df.sentence_idx.max()

1500.0

In [0]:
# class distribution
df.tag.value_counts(normalize=True )

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [0]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [0]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [0]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
0,1.0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48
1,1.0,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48
2,1.0,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48
3,1.0,9,through,32,marched,33,18,9,of,demonstrators,have,O,48
4,1.0,16,London,9,through,32,33,18,demonstrators,have,marched,O,48


In [0]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [0]:
!pip install lightgbm

    100% |████████████████████████████████| 1.2MB 16.7MB/s 


In [0]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [0]:
lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [50,75],
              "learning_rate" : [0.1, 0.5, 1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200, 300]
             }

In [0]:
columns = ['sentence_idx','next-next-pos','next-pos','pos','prev-pos','prev-prev-pos']

In [0]:
grid_search = GridSearchCV(lg,
                           n_jobs=-1,
                           param_grid=param_dist,
                           cv = 3,
                           verbose=5)

grid_search.fit(df_train[columns], y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300 
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300, score=0.7746083004425308, total= 7.1min
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300, score=0.960222514654863, total= 8.2min
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=300, score=0.6790308106491175, total= 5.5min
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900, score=0.8378184427700036, total= 6.5min
[CV] learning_rate=0.1, max_depth=50, n_estimators=200, num_leaves=900 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=200

[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 48.9min


[CV]  learning_rate=0.1, max_depth=50, n_estimators=300, num_leaves=900, score=0.8002991325157045, total=13.7min
[CV] learning_rate=0.1, max_depth=50, n_estimators=300, num_leaves=1200 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=300, num_leaves=1200, score=0.6967468006219352, total= 3.4min
[CV] learning_rate=0.1, max_depth=50, n_estimators=300, num_leaves=1200 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=300, num_leaves=1200, score=0.8294054312716832, total= 3.4min
[CV] learning_rate=0.1, max_depth=75, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.1, max_depth=75, n_estimators=200, num_leaves=300, score=0.7746083004425308, total= 1.9min
[CV] learning_rate=0.1, max_depth=75, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.1, max_depth=50, n_estimators=300, num_leaves=1200, score=0.8190846545019443, total= 4.9min
[CV] learning_rate=0.1, max_depth=75, n_estimators=200, num_leaves=300 
[CV]  learning_rate=0.1, max_depth=75, n_estimators=200, num_leaves=30

[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 127.9min


[CV]  learning_rate=0.5, max_depth=75, n_estimators=300, num_leaves=900, score=0.7562668262040084, total= 4.8min
[CV] learning_rate=0.5, max_depth=75, n_estimators=300, num_leaves=1200 
[CV]  learning_rate=0.5, max_depth=75, n_estimators=300, num_leaves=1200, score=0.7174979069489296, total= 4.4min
[CV] learning_rate=0.5, max_depth=75, n_estimators=300, num_leaves=1200 
[CV]  learning_rate=0.5, max_depth=75, n_estimators=300, num_leaves=1200, score=0.6705347529608805, total=  58.5s
[CV] learning_rate=1, max_depth=50, n_estimators=200, num_leaves=300 .
[CV]  learning_rate=0.5, max_depth=75, n_estimators=300, num_leaves=1200, score=0.7679329943164822, total=  57.8s
[CV] learning_rate=1, max_depth=50, n_estimators=200, num_leaves=300 .
[CV]  learning_rate=1, max_depth=50, n_estimators=200, num_leaves=300, score=0.7650998684367899, total=  40.1s
[CV] learning_rate=1, max_depth=50, n_estimators=200, num_leaves=300 .
[CV]  learning_rate=1, max_depth=50, n_estimators=200, num_leaves=300, scor

[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 152.6min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [50, 75], 'learning_rate': [0.1, 0.5, 1], 'num_leaves': [300, 900, 1200], 'n_estimators': [200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [0]:
print("Best parameters: {} with score: {}".format(grid_search.best_params_,
                                                  grid_search.best_score_))

Best parameters: {'learning_rate': 0.1, 'max_depth': 75, 'n_estimators': 300, 'num_leaves': 300} with score: 0.8331572126408134


In [0]:
model = grid_search.best_estimator_.fit(df_train[columns], y_train)

In [0]:
print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

train 0.07234807581842799
test 0.07518966533865713


In [0]:
!pip install catboost

    100% |████████████████████████████████| 55.3MB 430kB/s 


In [0]:
from catboost import CatBoostClassifier

In [0]:
param_dist = {'learning_rate': [0.01, 0.1, 1.0],
              'depth': [5, 10]}

cbc = CatBoostClassifier(learning_rate=1,
                         iterations=800,
                         random_state=SEED,
                         early_stopping_rounds=10,
                         loss_function='MultiClassOneVsAll',
                         custom_loss='F1')

grid_search_cat = GridSearchCV(cbc,
                           n_jobs=-1,
                           param_grid=param_dist,
                           cv = 3,
                           verbose=5)

grid_search_cat.fit(df_train[columns], y_train)

In [0]:
print("Best parameters: {} with score: {}".format(grid_search_cat.best_params_,
                                                  grid_search_cat.best_score_))

AttributeError: ignored

**Здесь у меня вылетел интернет и все слетело... Постараюсь прогнать GridSearch еще раз...**

